In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbs
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

housing = datasets.fetch_california_housing(as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(housing.data, housing.target, shuffle=True)

In [19]:
from sklearn.svm import SVR, LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

svr_reg = make_pipeline(StandardScaler(), SVR())
svr_scores = cross_val_score(svr_reg, X_train, y_train, cv=5)
svr_scores.mean()


0.7447301033940587

In [39]:
y_train.iloc[:2000, :]

IndexingError: Too many indexers

In [41]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

params = {
    'svr__kernel': ['poly', 'rbf'], 
    'svr__C': uniform(1,100),
    'svr__epsilon': uniform(0.1, 3),
    'svr__degree': randint(1, 10) 
}

svr_rnd_search = RandomizedSearchCV(svr_reg, param_distributions=params, n_iter=100)
svr_rnd_search.fit(X_train.iloc[:2000,:], y_train.iloc[:2000])
predictions = svr_rnd_search.predict(X_test)

RandomizedSearchCV(estimator=Pipeline(steps=[('standardscaler',
                                              StandardScaler()),
                                             ('svr', SVR())]),
                   n_iter=100,
                   param_distributions={'svr__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x284faafd0>,
                                        'svr__degree': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x284faa2e0>,
                                        'svr__epsilon': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x284fab670>,
                                        'svr__kernel': ['poly', 'rbf']})

In [45]:
svr_rnd_search.best_estimator_, svr_rnd_search.best_score_, predictions

(Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('svr',
                  SVR(C=11.950649162962367, degree=7,
                      epsilon=0.25100759101865666))]),
 0.7237567118902904,
 array([3.52661794, 1.39004871, 0.77924066, ..., 3.03816734, 1.58813705,
        1.46836179]))

In [46]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, predictions, squared=False)
rmse

0.5858048255167208